In [1]:
import pandas as pd

data = pd.read_csv('../csv/cleaned/cleaned_variations_review_posts.csv')
data

,tourist_destination,review_post,removed_special_charas_review_post,to_lower_case_review_post,removed_stop_words_review_post,lemmatized_words_review_post
0,Burnham Park,An excellent place for family and friends to v...,An excellent place for family and friends to v...,an excellent place for family and friends to v...,excellent place family friends visit entrance ...,excellent place family friend visit entrance f...
1,Burnham Park,A lovely place to be on a nice day.We were ple...,A lovely place to be on a nice day We were ple...,a lovely place to be on a nice day we were ple...,lovely place nice day pleasantly surprised ple...,lovely place nice day pleasantly surprised ple...
2,Burnham Park,Definitely one of the best places to visit in ...,Definitely one of the best places to visit in ...,definitely one of the best places to visit in ...,definitely best places visit baguio burnham pa...,definitely best place visit baguio burnham par...
3,Burnham Park,I have always been looking at this park ever s...,I have always been looking at this park ever s...,i have always been looking at this park ever s...,looking park chance review park flexed view pa...,looking park chance review park flexed view pa...
4,Burnham Park,Very beautiful park with boating and cycling a...,Very beautiful park with boating and cycling a...,very beautiful park with boating and cycling a...,beautiful park boating cycling available insid...,beautiful park boating cycling available insid...
...,...,...,...,...,...,...
7724,Teacher's Camp,Place is quite old but still works very well. ...,Place is quite old but still works very well ...,place is quite old but still works very well ...,place quite old works parking problem good pro...,place quite old work parking problem good prov...
7725,Teacher's Camp,Not crowded. Lots of open space.,Not crowded Lots of open space,not crowded lots of open space,crowded lots open space,crowded lot open space
7726,Teacher's Camp,Some staff are not approachable when i went to...,Some staff are not approachable when i went to...,some staff are not approachable when i went to...,staff approachable went make reservations nice...,staff approachable went make reservation nice ...
7727,Teacher's Camp,Very very affordable place. Nice and cozy. Kak...,Very very affordable place Nice and cozy Kak...,very very affordable place nice and cozy kak...,affordable place nice cozy kakatakot lang ng k...,affordable place nice cozy kakatakot lang ng k...


In [59]:
print(len(list(data['tourist_destination'].unique())))

18


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(ngram_range = (1, 1))
tfv_text = tfv.fit_transform(data['lemmatized_words_review_post'].values.astype('U'))

words = tfv.get_feature_names_out()

In [9]:
tfv_text

<7729x8677 sparse matrix of type '<class 'numpy.float64'>'
	with 97903 stored elements in Compressed Sparse Row format>

In [37]:
import gensim

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

words = list(sent_to_words(data['lemmatized_words_review_post'].values.tolist()))
print(words[0])

['excellent', 'place', 'family', 'friend', 'visit', 'entrance', 'fee', 'lake', 'ride', 'boat', 'park', 'ground', 'ride', 'small', 'bike', 'place', 'recommend', 'night', 'get', 'quite', 'dark', 'spot', 'morning', 'superb', 'experience', 'definitely', 'want', 'come']


In [39]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(words)
# Create Corpus
texts = words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]


In [40]:
dict_ = corpora.Dictionary(words)
print(dict_)

Dictionary(8647 unique tokens: ['bike', 'boat', 'come', 'dark', 'definitely']...)


In [45]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in words]
print(doc_term_matrix[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]


In [49]:
# Build LDA model
from gensim.models import CoherenceModel

lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=18, passes=20, 
random_state=20, chunksize = 10000, eval_every = 10, iterations = 100)



In [52]:
from pprint import pprint

# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.102*"strawberry" + 0.032*"farm" + 0.020*"picking" + 0.015*"ice" + '
  '0.015*"pick" + 0.015*"cream" + 0.014*"fresh" + 0.013*"lot" + 0.013*"taho" + '
  '0.012*"buy"'),
 (1,
  '0.019*"place" + 0.019*"snow" + 0.013*"really" + 0.012*"photo" + '
  '0.011*"worth" + 0.009*"kid" + 0.008*"view" + 0.008*"good" + 0.007*"like" + '
  '0.007*"just"'),
 (2,
  '0.062*"park" + 0.026*"baguio" + 0.023*"place" + 0.016*"city" + 0.015*"just" '
  '+ 0.014*"burnham" + 0.014*"boat" + 0.012*"ride" + 0.009*"bike" + '
  '0.009*"enjoy"'),
 (3,
  '0.052*"buy" + 0.032*"good" + 0.032*"pasalubong" + 0.031*"place" + '
  '0.025*"market" + 0.024*"product" + 0.022*"baguio" + 0.018*"fresh" + '
  '0.017*"price" + 0.017*"vegetable"'),
 (4,
  '0.026*"place" + 0.016*"pm" + 0.015*"night" + 0.012*"come" + 0.012*"people" '
  '+ 0.011*"early" + 0.009*"open" + 0.009*"picture" + 0.009*"experience" + '
  '0.009*"time"'),
 (5,
  '0.019*"christmas" + 0.017*"place" + 0.016*"great" + 0.015*"garden" + '
  '0.014*"crowded" + 0.01

In [51]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.5371774856512972
